## Regresie Liniară și Polinomială pentru Stabilirea Prețului Dovlecilor - Lecția 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografic de Dasani Madipalli</figcaption>


<!--![Infografic de Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Introducere

Până acum, ai explorat ce este regresia folosind date de probă din setul de date privind prețurile dovlecilor, pe care îl vom utiliza pe parcursul acestei lecții. De asemenea, l-ai vizualizat folosind `ggplot2`. 💪

Acum ești pregătit să aprofundezi regresia pentru ML. În această lecție, vei învăța mai multe despre două tipuri de regresie: *regresia liniară de bază* și *regresia polinomială*, împreună cu câteva aspecte matematice care stau la baza acestor tehnici.

> Pe parcursul acestui curriculum, presupunem cunoștințe minime de matematică și încercăm să o facem accesibilă pentru studenții care vin din alte domenii, așa că fii atent la notițe, 🧮 explicații, diagrame și alte instrumente de învățare care să te ajute să înțelegi mai bine.

#### Pregătire

Ca o reamintire, încarci aceste date pentru a pune întrebări despre ele.

-   Care este cel mai bun moment pentru a cumpăra dovleci?

-   Ce preț pot să mă aștept pentru o ladă de dovleci miniaturali?

-   Ar trebui să îi cumpăr în coșuri de jumătate de bushel sau în cutii de 1 1/9 bushel? Hai să continuăm să explorăm aceste date.

În lecția anterioară, ai creat un `tibble` (o reinterpretare modernă a cadrului de date) și l-ai populat cu o parte din setul de date original, standardizând prețurile pe bushel. Totuși, făcând asta, ai reușit să aduni doar aproximativ 400 de puncte de date și doar pentru lunile de toamnă. Poate putem obține mai multe detalii despre natura datelor printr-o curățare mai amănunțită? Vom vedea... 🕵️‍♀️

Pentru această sarcină, vom avea nevoie de următoarele pachete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) este o [colecție de pachete R](https://www.tidyverse.org/packages) concepută pentru a face știința datelor mai rapidă, mai ușoară și mai distractivă!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) este un cadru [format din pachete](https://www.tidymodels.org/packages/) pentru modelare și învățare automată.

-   `janitor`: Pachetul [janitor](https://github.com/sfirke/janitor) oferă instrumente simple pentru examinarea și curățarea datelor murdare.

-   `corrplot`: Pachetul [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) oferă un instrument vizual exploratoriu pentru matricea de corelație, care sprijină reordonarea automată a variabilelor pentru a ajuta la detectarea tiparelor ascunse între variabile.

Le poți instala astfel:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Scriptul de mai jos verifică dacă ai pachetele necesare pentru a finaliza acest modul și le instalează pentru tine în cazul în care lipsesc.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Vom încărca ulterior aceste pachete grozave și le vom face disponibile în sesiunea noastră curentă de R. (Aceasta este doar pentru ilustrare, `pacman::p_load()` a făcut deja acest lucru pentru tine)

## 1. O linie de regresie liniară

Așa cum ai învățat în Lecția 1, scopul unui exercițiu de regresie liniară este să poți trasa o *linie* *de* *cea mai bună potrivire* pentru a:

-   **Arăta relațiile dintre variabile**. Arată relația dintre variabile.

-   **Face predicții**. Realizează predicții precise despre unde ar putea să se încadreze un nou punct de date în raport cu acea linie.

Pentru a trasa acest tip de linie, folosim o tehnică statistică numită **Regresie prin metoda celor mai mici pătrate**. Termenul `cei mai mici pătrate` înseamnă că toate punctele de date din jurul liniei de regresie sunt ridicate la pătrat și apoi adunate. Ideal, suma finală este cât mai mică posibil, deoarece dorim un număr redus de erori, sau `cei mai mici pătrate`. Astfel, linia de cea mai bună potrivire este linia care ne oferă cea mai mică valoare pentru suma erorilor pătrate - de aici numele *regresie prin metoda celor mai mici pătrate*.

Facem acest lucru deoarece dorim să modelăm o linie care are cea mai mică distanță cumulativă față de toate punctele noastre de date. De asemenea, ridicăm termenii la pătrat înainte de a-i aduna, deoarece ne preocupă magnitudinea lor, nu direcția.

> **🧮 Arată-mi matematica**
>
> Această linie, numită *linia de cea mai bună potrivire*, poate fi exprimată prin [o ecuație](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` este '`variabila explicativă` sau `predictor`'. `Y` este '`variabila dependentă` sau `rezultat`'. Panta liniei este `b`, iar `a` este interceptul pe axa Y, care se referă la valoarea lui `Y` când `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "panta = $y/x$")
    Infografic de Jen Looper
>
> Mai întâi, calculează panta `b`.
>
> Cu alte cuvinte, referindu-ne la întrebarea originală despre datele dovleacului: "prezice prețul unui dovleac pe bucată în funcție de lună", `X` ar reprezenta prețul, iar `Y` ar reprezenta luna vânzării.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografic de Jen Looper
> 
> Calculează valoarea lui Y. Dacă plătești în jur de \$4, trebuie să fie aprilie!
>
> Matematica care calculează linia trebuie să demonstreze panta liniei, care depinde și de interceptul, sau unde se află `Y` când `X = 0`.
>
> Poți observa metoda de calcul pentru aceste valori pe site-ul [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Vizitează și [acest calculator pentru metoda celor mai mici pătrate](https://www.mathsisfun.com/data/least-squares-calculator.html) pentru a vedea cum valorile numerelor influențează linia.

Nu e chiar atât de înfricoșător, nu-i așa? 🤓

#### Corelație

Un alt termen de înțeles este **Coeficientul de Corelație** între variabilele X și Y date. Folosind un grafic de dispersie, poți vizualiza rapid acest coeficient. Un grafic cu puncte de date distribuite într-o linie ordonată are o corelație ridicată, dar un grafic cu puncte de date distribuite aleatoriu între X și Y are o corelație scăzută.

Un model bun de regresie liniară va fi unul care are un Coeficient de Corelație ridicat (mai aproape de 1 decât de 0) folosind metoda regresiei prin cei mai mici pătrate cu o linie de regresie.


## **2. Un dans cu datele: crearea unui cadru de date care va fi utilizat pentru modelare**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ilustrație de @allison_horst</figcaption>


<!--![Ilustrație de \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Încarcă bibliotecile necesare și setul de date. Convertește datele într-un cadru de date care conține un subset al datelor:

-   Selectează doar dovlecii prețuiți pe bușel

-   Convertește data într-o lună

-   Calculează prețul ca medie între prețurile maxime și minime

-   Convertește prețul pentru a reflecta prețul pe cantitatea de bușel

> Am acoperit acești pași în [lecția anterioară](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

În spiritul aventurii pure, să explorăm [`pachetul janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) care oferă funcții simple pentru examinarea și curățarea datelor murdare. De exemplu, să aruncăm o privire asupra numelor coloanelor pentru datele noastre:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Putem face mai bine. Haideți să transformăm aceste nume de coloane în `friendR` prin convertirea lor la convenția [snake_case](https://en.wikipedia.org/wiki/Snake_case) folosind `janitor::clean_names`. Pentru a afla mai multe despre această funcție: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Mult mai ordonat 🧹! Acum, un dans cu datele folosind `dplyr`, la fel ca în lecția anterioară! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bravo! 👌 Acum ai un set de date curat și ordonat pe care poți construi noul tău model de regresie!

Ce zici de un grafic scatter?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Un grafic de tip scatter ne amintește că avem date doar din lunile august până în decembrie. Probabil avem nevoie de mai multe date pentru a putea trage concluzii într-un mod liniar.

Să aruncăm din nou o privire asupra datelor noastre de modelare:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Ce-ar fi dacă am vrea să prezicem `price` (prețul) unui dovleac bazându-ne pe coloanele `city` (oraș) sau `package` (ambalaj), care sunt de tip caracter? Sau, și mai simplu, cum am putea găsi corelația (care necesită ca ambele intrări să fie numerice) între, de exemplu, `package` și `price`? 🤷🤷

Modelele de învățare automată funcționează cel mai bine cu caracteristici numerice, mai degrabă decât cu valori text, așa că, în general, trebuie să convertiți caracteristicile categorice în reprezentări numerice.

Aceasta înseamnă că trebuie să găsim o modalitate de a reformata predictorii noștri pentru a-i face mai ușor de utilizat de către un model, un proces cunoscut sub numele de `feature engineering`.


## 3. Preprocesarea datelor pentru modelare cu recipes 👩‍🍳👨‍🍳

Activitățile care reformatează valorile predictorilor pentru a le face mai ușor de utilizat de către un model sunt denumite `feature engineering`.

Modelele diferite au cerințe diferite de preprocesare. De exemplu, metoda celor mai mici pătrate necesită `codificarea variabilelor categorice`, cum ar fi luna, varietatea și city_name. Acest lucru implică pur și simplu `transformarea` unei coloane cu `valori categorice` într-una sau mai multe `coloane numerice` care înlocuiesc coloana originală.

De exemplu, să presupunem că datele tale includ următoarea caracteristică categorică:

|  city   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Poți aplica *codificarea ordinală* pentru a înlocui fiecare categorie cu o valoare întreagă unică, astfel:

| city |
|:----:|
|  0   |
|  1   |
|  2   |

Și exact asta vom face cu datele noastre!

În această secțiune, vom explora un alt pachet uimitor din Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - care este conceput pentru a te ajuta să preprocesezi datele **înainte** de antrenarea modelului. În esență, o rețetă este un obiect care definește ce pași ar trebui aplicați unui set de date pentru a-l pregăti pentru modelare.

Acum, să creăm o rețetă care pregătește datele noastre pentru modelare prin înlocuirea fiecărei observații din coloanele predictorilor cu un întreg unic:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Super! 👏 Tocmai am creat prima noastră rețetă care specifică un rezultat (prețul) și predictorii corespunzători, iar toate coloanele de predictori trebuie să fie codificate într-un set de numere întregi 🙌! Hai să o descompunem rapid:

- Apelul la `recipe()` cu o formulă indică rețetei *rolurile* variabilelor folosind datele `new_pumpkins` ca referință. De exemplu, coloana `price` a fost atribuită rolul de `outcome`, în timp ce restul coloanelor au fost atribuite rolul de `predictor`.

- `step_integer(all_predictors(), zero_based = TRUE)` specifică faptul că toți predictorii trebuie să fie convertiți într-un set de numere întregi, numerotarea începând de la 0.

Suntem siguri că te gândești: "Este atât de tare!! Dar ce fac dacă vreau să confirm că rețetele fac exact ceea ce mă aștept să facă? 🤔"

Este o idee grozavă! Vezi tu, odată ce rețeta ta este definită, poți estima parametrii necesari pentru a preprocesa efectiv datele și apoi să extragi datele procesate. De obicei, nu trebuie să faci acest lucru când folosești Tidymodels (vom vedea convenția normală în doar un minut-\> `workflows`), dar poate fi util atunci când vrei să faci un fel de verificare pentru a confirma că rețetele funcționează conform așteptărilor.

Pentru asta, vei avea nevoie de două verbe suplimentare: `prep()` și `bake()`, iar ca întotdeauna, micii noștri prieteni din R de la [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) te ajută să înțelegi mai bine acest lucru!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustrație de @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): estimează parametrii necesari dintr-un set de antrenament care pot fi aplicați ulterior altor seturi de date. De exemplu, pentru o anumită coloană predictor, ce observație va fi atribuită ca întreg 0 sau 1 sau 2 etc.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): ia o rețetă pregătită și aplică operațiunile pe orice set de date.

Așadar, să pregătim și să aplicăm rețetele noastre pentru a confirma cu adevărat că, în spatele cortinei, coloanele predictor vor fi mai întâi codificate înainte ca un model să fie ajustat.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Ura! 🥳 Datele procesate `baked_pumpkins` au toți predictorii codificați, confirmând că pașii de preprocesare definiți ca rețeta noastră funcționează așa cum ne așteptam. Acest lucru face ca datele să fie mai greu de citit pentru tine, dar mult mai ușor de înțeles pentru Tidymodels! Ia-ți puțin timp să descoperi ce observație a fost mapată la un întreg corespunzător.

De asemenea, merită menționat că `baked_pumpkins` este un cadru de date pe care putem efectua calcule.

De exemplu, să încercăm să găsim o corelație bună între două puncte din datele tale pentru a construi un model predictiv eficient. Vom folosi funcția `cor()` pentru asta. Tastează `?cor()` pentru a afla mai multe despre funcție.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Se pare că există doar o corelație slabă între Oraș și Preț. Totuși, există o corelație ceva mai bună între Pachet și Prețul acestuia. Are sens, nu-i așa? De obicei, cu cât cutia de produse este mai mare, cu atât prețul este mai ridicat.

De vreme ce suntem aici, hai să încercăm să vizualizăm și o matrice de corelație pentru toate coloanele folosind pachetul `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Mult mai bine.

O întrebare bună de pus acum despre aceste date ar fi: '`Ce preț pot să mă aștept pentru un pachet de dovleci?`' Hai să intrăm direct în subiect!

> Notă: Când **`bake()`** rețeta pregătită **`pumpkins_prep`** cu **`new_data = NULL`**, extragi datele procesate (adică codificate) de antrenament. Dacă ai avea un alt set de date, de exemplu un set de testare, și ai vrea să vezi cum o rețetă l-ar pre-procesa, ai coace pur și simplu **`pumpkins_prep`** cu **`new_data = test_set`**

## 4. Construiește un model de regresie liniară

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografic de Dasani Madipalli</figcaption>


<!--![Infografic de Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Acum că am construit o rețetă și am confirmat că datele vor fi pre-procesate corespunzător, să construim un model de regresie pentru a răspunde la întrebarea: `Ce preț pot să mă aștept pentru un pachet de dovleac dat?`

#### Antrenează un model de regresie liniară folosind setul de antrenament

După cum probabil ai dedus deja, coloana *price* este variabila `rezultat`, în timp ce coloana *package* este variabila `predictor`.

Pentru a face acest lucru, mai întâi vom împărți datele astfel încât 80% să fie utilizate pentru antrenament și 20% pentru setul de testare, apoi vom defini o rețetă care va codifica coloana predictor într-un set de numere întregi, iar apoi vom construi o specificație de model. Nu vom pregăti și coace rețeta noastră, deoarece știm deja că va pre-procesa datele conform așteptărilor.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bravo! Acum că avem o rețetă și o specificație a modelului, trebuie să găsim o modalitate de a le combina într-un obiect care să preproceseze mai întâi datele (prep+bake în fundal), să ajusteze modelul pe datele preprocesate și să permită, de asemenea, activități potențiale de post-procesare. Cum îți sună asta pentru liniștea ta sufletească!🤩

În Tidymodels, acest obiect convenabil se numește [`workflow`](https://workflows.tidymodels.org/) și reunește în mod convenabil componentele tale de modelare! Acesta este ceea ce am numi *pipelines* în *Python*.

Așadar, să împachetăm totul într-un workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 În plus, un flux de lucru poate fi ajustat/antrenat în mod similar cu un model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Din rezultatele modelului, putem observa coeficienții învățați în timpul antrenării. Aceștia reprezintă coeficienții liniei de ajustare optimă care ne oferă cea mai mică eroare totală între variabila reală și cea prezisă.

#### Evaluarea performanței modelului folosind setul de testare

Este momentul să vedem cum s-a descurcat modelul 📏! Cum facem acest lucru?

Acum că am antrenat modelul, îl putem folosi pentru a face predicții pentru `test_set` utilizând `parsnip::predict()`. Apoi putem compara aceste predicții cu valorile reale ale etichetelor pentru a evalua cât de bine (sau nu!) funcționează modelul.

Să începem prin a face predicții pentru setul de testare, apoi să legăm coloanele la setul de testare.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Da, tocmai ai antrenat un model și l-ai folosit pentru a face predicții!🔮 Este bun? Hai să evaluăm performanța modelului!

În Tidymodels, facem acest lucru folosind `yardstick::metrics()`! Pentru regresia liniară, să ne concentrăm pe următoarele metrici:

-   `Root Mean Square Error (RMSE)`: Rădăcina pătrată a [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Aceasta oferă o metrică absolută în aceeași unitate ca eticheta (în acest caz, prețul unui dovleac). Cu cât valoarea este mai mică, cu atât modelul este mai bun (într-un sens simplist, reprezintă prețul mediu cu care predicțiile sunt greșite!).

-   `Coefficient of Determination (cunoscut de obicei ca R-squared sau R2)`: O metrică relativă în care, cu cât valoarea este mai mare, cu atât modelul se potrivește mai bine. În esență, această metrică reprezintă cât de mult din variația dintre valorile etichetei prezise și cele reale modelul reușește să explice.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Acolo se duce performanța modelului. Să vedem dacă putem obține o indicație mai bună vizualizând un grafic de dispersie al pachetului și prețului, apoi folosind predicțiile făcute pentru a suprapune o linie de ajustare optimă.

Aceasta înseamnă că va trebui să pregătim și să procesăm setul de testare pentru a codifica coloana pachet, apoi să legăm acest lucru de predicțiile făcute de modelul nostru.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Super! După cum se poate observa, modelul de regresie liniară nu generalizează foarte bine relația dintre un pachet și prețul său corespunzător.

🎃 Felicitări, tocmai ai creat un model care poate ajuta la prezicerea prețului pentru câteva varietăți de dovleci. Grădina ta de dovleci pentru sărbători va fi minunată. Dar probabil poți crea un model mai bun!

## 5. Construiește un model de regresie polinomială

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografic de Dasani Madipalli</figcaption>


<!--![Infografic de Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Uneori, datele noastre pot să nu aibă o relație liniară, dar totuși dorim să prezicem un rezultat. Regresia polinomială ne poate ajuta să facem predicții pentru relații mai complexe, non-liniare.

Luați, de exemplu, relația dintre ambalaj și preț în setul nostru de date despre dovleci. Deși uneori există o relație liniară între variabile - cu cât dovleacul este mai mare ca volum, cu atât prețul este mai mare - alteori aceste relații nu pot fi reprezentate ca un plan sau o linie dreaptă.

> ✅ Iată [câteva exemple suplimentare](https://online.stat.psu.edu/stat501/lesson/9/9.8) de date care ar putea utiliza regresia polinomială
>
> Priviți din nou relația dintre varietate și preț în graficul anterior. Pare că acest scatterplot ar trebui analizat neapărat printr-o linie dreaptă? Poate că nu. În acest caz, puteți încerca regresia polinomială.
>
> ✅ Polinoamele sunt expresii matematice care pot consta din una sau mai multe variabile și coeficienți

#### Antrenează un model de regresie polinomială folosind setul de antrenament

Regresia polinomială creează o *linie curbată* pentru a se potrivi mai bine datelor non-liniare.

Să vedem dacă un model polinomial va performa mai bine în realizarea predicțiilor. Vom urma un proces oarecum similar cu cel de dinainte:

-   Creează o rețetă care specifică pașii de preprocesare ce trebuie efectuați asupra datelor noastre pentru a le pregăti pentru modelare, adică: codificarea predictorilor și calcularea polinoamelor de gradul *n*

-   Construiește o specificație de model

-   Combină rețeta și specificația modelului într-un flux de lucru

-   Creează un model prin ajustarea fluxului de lucru

-   Evaluează cât de bine performează modelul pe datele de testare

Să trecem direct la treabă!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evaluează performanța modelului

👏👏 Ai construit un model polinomial, hai să facem predicții pe setul de testare!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Ura, să evaluăm cum s-a descurcat modelul pe test_set folosind `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Performanță mult mai bună.

`rmse` a scăzut de la aproximativ 7 la aproximativ 3, ceea ce indică o reducere a erorii dintre prețul real și prețul prezis. Poți interpreta *în linii mari* acest lucru ca însemnând că, în medie, predicțiile incorecte sunt greșite cu aproximativ 3$. `rsq` a crescut de la aproximativ 0.4 la 0.8.

Toate aceste metrici indică faptul că modelul polinomial funcționează mult mai bine decât modelul liniar. Bravo!

Hai să vedem dacă putem vizualiza acest lucru!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Poți vedea o linie curbată care se potrivește mai bine cu datele tale! 🤩

Poți face aceasta mai netedă prin transmiterea unei formule polinomiale către `geom_smooth` astfel:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

La fel de lin ca o curbă!🤩

Iată cum poți face o nouă predicție:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Predicția modelului `polynomial` are sens, având în vedere graficele de dispersie ale `price` și `package`! Și, dacă acesta este un model mai bun decât cel anterior, privind aceleași date, trebuie să bugetezi pentru aceste dovleci mai scumpi!

🏆 Bravo! Ai creat două modele de regresie într-o singură lecție. În secțiunea finală despre regresie, vei învăța despre regresia logistică pentru a determina categorii.

## **🚀Provocare**

Testează mai multe variabile diferite în acest notebook pentru a vedea cum corelația corespunde cu acuratețea modelului.

## [**Quiz după lecție**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Recapitulare & Studiu individual**

În această lecție am învățat despre regresia liniară. Există și alte tipuri importante de regresie. Citește despre tehnicile Stepwise, Ridge, Lasso și Elasticnet. Un curs bun pentru a învăța mai multe este [cursul de Învățare Statistică de la Stanford](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Dacă vrei să afli mai multe despre cum să folosești cadrul uimitor Tidymodels, te rugăm să consulți următoarele resurse:

-   Website-ul Tidymodels: [Începe cu Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn și Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **MULȚUMIRI SPECIALE:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) pentru crearea ilustrațiilor uimitoare care fac R mai primitor și captivant. Găsește mai multe ilustrații în [galeria ei](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
